In [ ]:
import numpy as np
import onnx
from onnx import numpy_helper
import inspect

In [ ]:
# Define a dictionary

def MatMul(input, initializer):
    return np.dot(input, initializer)

def Mul(input, initializer): 
    return np.dot(input, initializer)

def Add(input, initializer):
    return input + initializer

def Transpose(input, perm): 
    return np.transpose(input, perm)
    
def Conv(x, w, b, dilations, group, kernel_shape, pads, strides): 
    batch_size, in_channels, in_height, in_width = x.shape
    out_channels, _, kernel_height, kernel_width = w.shape
    stride_height, stride_width = strides
    pad_height_begin, pad_width_begin, pad_height_end, pad_width_end = pads

    out_height = (in_height + pad_height_begin + pad_height_end - kernel_height) // stride_height + 1
    out_width = (in_width + pad_width_begin + pad_width_end - kernel_width) // stride_width + 1

    y = np.zeros((batch_size, out_channels, out_height, out_width), dtype=np.float32)
    
    x_padded = np.pad(x, ((0, 0), (0, 0), (pad_height_begin, pad_height_end), (pad_width_begin, pad_width_end)), mode='constant')
    
    for i in range(out_height):
        for j in range(out_width):
            h_start = i * stride_height
            h_end = h_start + kernel_height
            w_start = j * stride_width
            w_end = w_start + kernel_width
            x_slice = x_padded[:, :, h_start:h_end, w_start:w_end]
            for k in range(out_channels):
                y[:, k, i, j] = np.sum(x_slice * w[k, :, :, :], axis=(1, 2, 3))
                
    if b is not None:
        y += b.reshape(1, -1, 1, 1)
        
    return y

def Relu(input): 
    return np.maximum(0, input)

def MaxPool(x, kernel_shape, strides, pads = [0, 0, 0, 0]): 
    batch_size, in_channels, in_height, in_width = x.shape
    kernel_height, kernel_width = kernel_shape
    stride_height, stride_width = strides
    pad_height_begin, pad_width_begin, pad_height_end, pad_width_end = pads

    out_height = (in_height + pad_height_begin + pad_height_end - kernel_height) // stride_height + 1
    out_width = (in_width + pad_width_begin + pad_width_end - kernel_width) // stride_width + 1

    y = np.zeros((batch_size, in_channels, out_height, out_width), dtype=np.float32)

    x_padded = np.pad(x, ((0, 0), (0, 0), (pad_height_begin, pad_height_end), (pad_width_begin, pad_width_end)), mode='constant')

    for i in range(out_height):
        for j in range(out_width):
            h_start = i * stride_height
            h_end = h_start + kernel_height
            w_start = j * stride_width
            w_end = w_start + kernel_width
            x_slice = x_padded[:, :, h_start:h_end, w_start:w_end]
            y[:, :, i, j] = np.max(x_slice, axis=(2, 3))

    return y

def Reshape(input, shape): 
    return np.reshape(input, shape)

def Gemm(A, B, C): 
    return np.dot(A, B) + C

function_dict = {
    "MatMul": MatMul,
    "Mul": Mul, 
    "Add": Add, 
    "Transpose": Transpose, 
    "Conv": Conv, 
    "Relu": Relu, 
    "MaxPool": MaxPool, 
    "Reshape": Reshape, 
    "Gemm": Gemm
    }

In [ ]:
def onnx_run(model, input):
    operators = []
    inputs = []
    initializers = []

    # get onnx operators

    operators = [node.op_type for node in model.graph.node]


    # get onnx initializers

    inputs = [i.input[1:] if len(i.input) > 1 else ['None'] for i in model.graph.node]

    # Create a dictionary to map initializer names to their array representations
    init_dict = {inits.name: numpy_helper.to_array(inits) for inits in model.graph.initializer}

    # Iterate over each list in inputs and map names to their initializers
    for inputss in inputs:
        initials = [init_dict[i] for i in inputss if i in init_dict]
        initializers.append(initials)


    # run operators
    result = input
    
    for operator, args, i in zip(operators, initializers, range(len(model.graph.node))): 

        # get onnx attributes

        # Get the function's parameter names
        attr = {attr.name: attr.ints for attr in model.graph.node[i].attribute}
        
        # Filter the params dictionary to only include the parameters that the function accepts
        func_params = inspect.signature(function_dict[operator]).parameters
        filtered_attrs = {k: v for k, v in attr.items() if k in func_params}

        result = function_dict[operator](result, *args, **filtered_attrs)
    
    return result

In [ ]:
simple_cnn = onnx.load("sequential_2.onnx")

import tensorflow as tf
flower_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/4/4e/Bridal_pink_-_morwell_rose_garden.jpg/800px-Bridal_pink_-_morwell_rose_garden.jpg"
flower_path = tf.keras.utils.get_file(origin = flower_url)

img = tf.keras.utils.load_img(
    flower_path, target_size = (180, 180)
)
img_array = tf.keras.utils.img_to_array(img)
input = tf.expand_dims(img_array, 0)


In [ ]:
onnx_run(simple_cnn, input)